In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import os
from neo_importers.neo_dapsys_importer import _do_files_need_fixing, _fix_separator_decimal_matching

file_path = Path("..")/".."/"Files"/"dapsys_crossing_tracks"/"fixed"

if _do_files_need_fixing(file_path):
    fixed_path = os.path.join(file_path, "fixed")
    _fix_separator_decimal_matching(in_path = file_path, out_path = fixed_path)
    file_path = fixed_path

In [62]:
%load_ext autoreload
%autoreload 2
from neo_importers.neo_dapsys_importer import import_dapsys_csv_files
from neo.core import Block
from neo_importers.neo_wrapper import MNGRecording

block: Block = import_dapsys_csv_files(directory = file_path, sampling_rate = "imply")
recording: MNGRecording = MNGRecording(block.segments[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Processing AP Track 1 with 618 latencies.: 100%|█████████████████████████████████████| 618/618 [00:06<00:00, 88.45it/s]


In [63]:
%autoreload 2
from features import FeatureDatabase, ResponseLatencyFeatureExtractor, NormalizedSignalEnergyExtractor, SpikeCountExtractor, AdaptiveSpikeCountExtractor
from quantities import second

# define feature db
db = FeatureDatabase(Path("../../Files/dapsys_crossing_tracks/features"), recording)

try: # check if we can just load the features somehow
    db.load()
    print("Found feature database and loaded from files...")
except Exception as ex: # extract the features from the data base
    ap_channels = list(recording.action_potential_channels.keys())
    db.extract_features(ap_channels, ResponseLatencyFeatureExtractor, stimulus_channel = "es.0")
    db.extract_features(ap_channels, NormalizedSignalEnergyExtractor)
    db.extract_features(ap_channels, SpikeCountExtractor, timeframe = 100 * second, num_intervals = 8)
    db.extract_features(ap_channels, AdaptiveSpikeCountExtractor, timeframe = 100 * second, num_intervals = 6)
    db.store()
    print("Created and stored new feature db...")

Found feature database and loaded from files...


In [67]:
%autoreload 2
from plotting import FallingLeafPlot
from quantities import second, minute

flp = FallingLeafPlot(recording = recording, feature_db = db)

flp.plot(action_potential_channels = "all", plot_raw_signal = True, analog_signal_channel = "rd.1", \
         t_start = 1 * minute, post_stimulus_timeframe = 0.1 * second, num_intervals = 25, 
         ap_tracks = [])